# Introduction

This notebook contains the actual runnable Twitter stream! **Jump to section 2** to avoid reading all the setup. *(Section 1 contains library imports, neural network and tokenizer imports, and a handful of functions that are necessary for processing tweets and  making predictions. Nothing that wasn't already touched on in other notebooks.)*

## Libraries

In [17]:
from twython import Twython, TwythonStreamer
import time

# Below is all just for neural network

import string
import re

import nltk
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize

from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding
from keras.layers import Dropout, Activation, GlobalMaxPool1D
from keras.models import Sequential
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing import text, sequence
from keras.optimizers import Adam

import pickle

## Twython Object

We instantiate a Twython object that stores the authentication data required to run the following `Conversation` class in conjunction with the Twitter API.

In [10]:
# Set API version to 1.1. Version 2 doesn't have DM support yet.

t = Twython(app_key=TWITTER_APP_KEY, 
            app_secret=TWITTER_APP_KEY_SECRET, 
            oauth_token=TWITTER_ACCESS_TOKEN, 
            oauth_token_secret=TWITTER_ACCESS_TOKEN_SECRET,
            api_version='1.1')

## `Conversation` Class

A full explanation of this class is available in the notebook `twitter_chatbot.ipynb`. In brief, it allows for DM interactions with one Twitter user to be contained within a single instance Python class.

In [11]:
class Conversation:
    
    '''
    Class designed to be a representation of a unique conversation with one
    individual. Takes in a Twython instance, the unique user ID, and a brand
    that the bot is currently representing.
    
    Contains functions for storing, cleaning, and replying to messages.
    ---
    t
        Twython instance.
    user_id
        Twitter user ID. String or int.
    brand
        Defaults to Amazon, but can be any string.
    help_count
        Number of times Twitter user has sent "HELP" to chatbot. If count
        is >= 3, bot sets no_contact to True.
    no_contact
        Boolean. When True, bot will not reply to user.
    passed_to_human
        Whether or not the interaction has been passed on to a human employee.
        Defaults to False. When a user says "YES" they want to work with brand,
        this switches to True.
    '''
    
    def __init__(self, t, user_id, brand = 'Amazon'):
        self.t = t
        self.user_id = str(user_id)
        self.brand = brand
        self.help_count = 0
        self.messages = None
        self.no_contact = False
        self.passed_to_human = False
    
    
    
    def store_messages(self):

        '''
        Gets and saves all messages sent from Twitter user self.user_id to
        the bot. Output returns a list of dictionaries, where all keys and
        values are strings.
        '''
        
        messages = self.t.get_direct_messages()
        
        message_list = []

        for i in range(len(messages['events'])):

            message = messages['events'][i]
            sender_id = message['message_create']['sender_id']

            if sender_id == self.user_id:
                message_dict = {}
                message_dict['time'] = message['created_timestamp']
                message_dict['user_id'] = message['message_create']['sender_id']
                message_dict['text'] = message['message_create']['message_data']['text']
                message_list.append(message_dict)

        self.messages = message_list
        
        

    def clean_message(self, message):

        '''
        Cleans out punctuation and capitals from a user message. Returns the
        string as a list of strings so that replies can be parsed easily.
        ---
        message
            Must be a string, ideally as seen in store_messages()[i]['text']
        '''

        allowed_replies = ['yes', 'no', 'help', 'stop']

        for i in string.punctuation:
            message = message.replace(i, '').lower()

        new_message = []
        for word in message.split():
            if word in allowed_replies:
                new_message.append(word)

        return new_message

    

    def send_message(self, text):

        '''
        Sends a message to a specified user.
        ---
        text
            String of what to send to user.
        '''

        if self.no_contact == True:
            return 
        
        self.t.send_direct_message(
            event = {"type" : "message_create",
                     "message_create" : {"target": {"recipient_id" : self.user_id},
                                         "message_data" : {"text" : text}}}
        )
        


    def reply(self, message_in):
        
        '''
        This function allows the bot to talk back to users. It replies to four
        possible, pre-specified inputs, and provides an alternative for when the
        user input is not in the pre-specified list.
        ---
        message_in
            Must be a string. This is what the bot replies to.
        '''

        message_in = self.clean_message(message_in)
        allowed_replies = ['yes', 'no', 'help', 'stop']
        
        if self.no_contact == True:
            return
        
        if (len(message_in) > 1) or (len(message_in) == 0):
            if self.help_count >= 3:
                self.send_message("Looks like you're havin a hard' time bud. I'll leave you alone :/")
                self.no_contact = True
            else:
                self.send_message("I'm sorry but I don't know what you want. Please reply only with YES, NO, HELP, or STOP.")
            self.help_count += 1
            return

        if message_in[0] == 'yes':
            self.send_message("We're happy to hear it! A spokesperson will be in touch shortly :)")
            print(f'User at ID {self.user_id} is interested in collaborating! Get a human on this task at once!')
            self.passed_to_human = True
            return

        if message_in[0] == 'no':
            self.send_message("We're sad you won't be joining us. Have a nice day!")
            return

        if message_in[0] == 'help':
            self.send_message("Reply YES to show interest in a brand deal, NO to decline, HELP to see this message, and STOP to be put on our no-contact list.")
            self.help_count += 1
            return

        if message_in[0] == 'stop':
            self.no_contact = True
            return
    

    
    def greet(self):
        
        '''
        Greets a user with the chatbot, introduces what brand the bot works for,
        and offers four options for replying to the bot.
        ---
        All parameters pre-determined in __init__.
        '''
        
        text = f'Hi there! Wanna collaborate with {self.brand}? Please respond with YES, NO, HELP, or STOP.'
        self.send_message(text)

## Neural Network And Sentiment Prediction

Here we're loading in the neural network trained in `models.ipynb` as well as defining a few functions for cleaning tweets so that the model can predict on them. They're almost the same functions as used in `models.ipynb` for preprocessing data, just slightly changed to handle single data points instead.

In [12]:
model = keras.models.load_model('models/stem_model_5')

### `stem_clean()`

In [13]:
ps = PorterStemmer()

In [14]:
def stem_clean(text):
    '''
    Takes in a piece of text and cleans and stems it. Removes all punctuation,
    URL's, usernames, and hashtags. Sets everything to lowercase, tokenizes,
    removes stopwords, stems and returns it as a list of tokens.
    ---
    text
        String input to be cleaned.
    '''
    stop_words = stopwords.words("english")
    
    text = re.sub('@\S+', '', text)
    text = re.sub('http\S+', '', text)
    text = re.sub('#\S+', '', text)
    for i in string.punctuation:
        text = text.replace(i, '').lower()
    
    tokens = nltk.word_tokenize(text)
    new_tokens = []
    for token in tokens:
        if token.lower() not in stop_words:
            new_tokens.append(ps.stem(token))
            
    return new_tokens

In [18]:
with open('models/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

### `tweet_to_sequence()`

In [43]:
def tweet_to_sequence(tweet):
    '''
    Takes in a tweet and returns a padded tokenized array.
    ---
    tweet
        Tweet that has been cleaned by stem_clean(), ideally. Otherwise a list
        of token strings works, but less optimized.
    '''
    tokenized_tweet = tokenizer.texts_to_sequences([tweet])
    tweet_seq = sequence.pad_sequences(tokenized_tweet, maxlen = 45)
# Needs o index because returns an array of arrays and we only want the one
    return tweet_seq[0]

### `tweet_sentiment()`

In [73]:
def tweet_sentiment(tweet, model):
    '''
    Takes in a string and predicts sentiment.
    ---
    tweet
        Tweet that has been cleaned by stem_clean(), ideally. Otherwise a list
        of token strings works, but less optimized.
    model
        Trained neural network.
    '''
    tweet_seq = tweet_to_sequence(tweet)

    preds = model.predict(np.array([tweet_seq]))

# Needs 0 index because built for many but only want the first
    pred = list(preds[0])
    max_pred = max(pred)

    return pred.index(max_pred) + 1

Example of prediction. Value `3` means positive:

In [95]:
tweet_sentiment("Best belive imma take advantage of my amazon prime account", model)

3

# Twython Streaming

The moment we've all been waiting for! **This `Stream` can be set to run on its own,** and will wait until a user with at least a thousand followers tweets positively about Amazon. Then, **it initializes a `Conversation`** with that user and greets them. It **waits** for two minutes and **checks for a response.** (Twitter's API doesn't update DM information quickly enough to be faster than this, at least not in the free version). If the user has sent a message, the `Conversation` bot will **reply,** and the **process repeats.**

The bot's operation interrupts the `Stream` and no more searching occurs until the `Conversation` is over. In the future, I intend to learn more about **asynchronous execution** so that I can have a **master `Stream`** always running, with numerous `Conversation` bots assigned to different users as they appear.

## `Stream` Class

This class is based on the default `MyStreamer` class provided in the [Twython documentation](https://twython.readthedocs.io/en/latest/usage/streaming_api.html). I've immensely changed the method `on_success()` such that it intitializes and has a `Conversation` with the user.

In [10]:
class Stream(TwythonStreamer):

    def on_success(self, data):
        if data['user']['followers_count'] > 1000:
            if tweet_sentiment(data['text']) == 3:
                c = Conversation(user_id = data['id'], t = t)
                c.greet()
                while (c.no_contact == False) and (c.passed_to_human == False):
                    time.sleep(120)
                    c.store_messages()
                    if c.messages != None:
                        newest_message = c.messages[0]['text']
                        c.reply(newest_message)

    def on_error(self, status_code, data):
        print(status_code)
        self.disconnect()

## Stream Example

In [44]:
stream = Stream(TWITTER_APP_KEY, TWITTER_APP_KEY_SECRET,
                TWITTER_ACCESS_TOKEN, TWITTER_ACCESS_TOKEN_SECRET)

In [1]:
stream.statuses.filter(language = 'en',
                       track = '@amazon',
                       tweet_mode = 'extended')

In practice, the cell above outputs a user ID whenever someone agrees to work with the brand.